In [1]:
import sys
import time
import os.path
from glob import glob
from datetime import datetime, timedelta

# data tools
import h5py
import pygrib
import numpy as np
import numba as nb
import pandas as pd
import netCDF4 as nc

# custom tools
sys.path.insert(0, '/glade/u/home/ksha/PUBLISH/WFRT-PP-DEV/')
sys.path.insert(0, '/glade/u/home/ksha/PUBLISH/WFRT-PP-DEV/libs/')

import utils

# !!!! <---- change to your namelist
from namelist_casper import * 
from namelist_plot import *
import plot_lib as plib

In [2]:
import importlib
importlib.reload(utils)

<module 'utils' from '/glade/u/home/ksha/PUBLISH/WFRT-PP-DEV/libs/utils.py'>

In [29]:
importlib.reload(plib)

<module 'plot_lib' from '/glade/u/home/ksha/PUBLISH/WFRT-PP-DEV/libs/plot_lib.py'>

In [31]:
#importlib.reload(namelist_plot)
from namelist_plot import *

In [5]:
# graph tools
import cmaps
import cartopy.crs as ccrs
import cartopy.mpl.geoaxes
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection

from matplotlib import ticker
import matplotlib.ticker as mticker
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

%matplotlib inline

In [6]:
need_publish = False

# True: publication quality figures
# False: low resolution figures in the notebook

if need_publish:
    dpi_ = fig_keys['dpi']
else:
    dpi_ = 75

**Datetime**

In [7]:
dt_fmt = '20220413'
if dt_fmt == 'auto':
    dt_utc_now = datetime.utcnow()
    dt_fmt_string = datetime.strftime(dt_utc_now, '%Y%m%d')
else:
    dt_fmt_string = dt_fmt
    dt_utc_now = datetime.strptime(dt_fmt_string, '%Y%m%d')

**Geo-data**

In [8]:
with h5py.File(path_domain_namelist, 'r') as h5io:
    lat_bc = h5io['bc_lat'][...] # lats of the BC domain
    lon_bc = h5io['bc_lon'][...] # lons of the BC domain
    lon_4km = h5io['lon_4km'][...]
    lat_4km = h5io['lat_4km'][...]
    land_mask_bc = h5io['land_mask_bc'][...] # selecting OCEAN grids from the BC domain
    land_mask_bc_4km = h5io['land_mask_bc_4km'][...]
# ocean_mask_bc = np.logical_not(land_mask_bc) # selecting LAND grids from the BC domain
grid_shape = lon_bc.shape

# US states and CAN-US boundary
PROVINCE = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale=scale_param,
    facecolor='none')


In [9]:
name_list = ['United States of America']
geom_US = plib.get_country_geom(name_list, scale_param)

**GEFS outputs**

In [10]:
output_dir = output_dir_namelist.format(dt_fmt_string)
name_output = filename_CNN_output_namelist.format(dt_fmt_string)

with h5py.File(output_dir+name_output, 'r') as h5io:
    CNN_output = h5io['gefs_apcp'][...]
    
CNN_output[..., land_mask_bc] = np.nan

In [11]:
GEFS_raw = np.empty((ensemble_mumberp_raw_gefs_namelist+1, N_leads_namelist)+grid_shape)

# GEFS file path creation
GEFS_dir_base = path_gefs_member_namelist.format(dt_fmt_string)

for i, lead_ in enumerate(FCSTs_namelist):

    #print("Process lead time ind: {}".format(i))
    
    for member_ in range(ensemble_mumberp_raw_gefs_namelist+1):
        
        if member_ == 0:
            filename_ = filename_memberc_namelist.format(member_, int(lead_))
        else:
            filename_ = filename_memberp_namelist.format(member_, int(lead_))
    
        GEFS_dir_full = GEFS_dir_base+filename_

        with pygrib.open(GEFS_dir_full) as grb_io:
            grb_reader_apcp = grb_io.select(name='Total Precipitation')[0]
            apcp, _, _ = grb_reader_apcp.data(lat1=48.25, lat2=60.00, lon1=-141.0+360, lon2=-113.25+360)
        
        apcp = np.flipud(apcp)
        GEFS_raw[member_, i, ...] = apcp

Process lead time ind: 0
Process lead time ind: 1
Process lead time ind: 2
Process lead time ind: 3
Process lead time ind: 4
Process lead time ind: 5
Process lead time ind: 6
Process lead time ind: 7
Process lead time ind: 8
Process lead time ind: 9
Process lead time ind: 10
Process lead time ind: 11
Process lead time ind: 12
Process lead time ind: 13
Process lead time ind: 14
Process lead time ind: 15
Process lead time ind: 16
Process lead time ind: 17
Process lead time ind: 18
Process lead time ind: 19
Process lead time ind: 20
Process lead time ind: 21
Process lead time ind: 22
Process lead time ind: 23
Process lead time ind: 24
Process lead time ind: 25
Process lead time ind: 26
Process lead time ind: 27
Process lead time ind: 28
Process lead time ind: 29
Process lead time ind: 30
Process lead time ind: 31
Process lead time ind: 32
Process lead time ind: 33
Process lead time ind: 34
Process lead time ind: 35
Process lead time ind: 36
Process lead time ind: 37
Process lead time ind:

In [12]:
GEFS_raw[..., land_mask_bc] = np.nan

In [13]:
DATA = {}

In [14]:
LEADs_3H_ind = np.arange(0, N_leads_namelist, dtype=np.int)
LEADs_3H_hrs = np.arange(9.0, 24*7+3, 3)[:N_leads_namelist]

DATA['CNN_3_P10'] = np.quantile(CNN_output, 0.1, axis=0)
DATA['CNN_3_P50'] = np.quantile(CNN_output, 0.5, axis=0)
DATA['CNN_3_P90'] = np.quantile(CNN_output, 0.9, axis=0)

DATA['GEFS_3_P10'] = np.quantile(GEFS_raw, 0.1, axis=0)
DATA['GEFS_3_P50'] = np.quantile(GEFS_raw, 0.5, axis=0)
DATA['GEFS_3_P90'] = np.quantile(GEFS_raw, 0.9, axis=0)

In [15]:
accum_window = 8 # 8x3h = 1 day
output_freq = 2 # 2x3h = 6h per output
skip_start = 1 # skip the first 3hr, start from 12hr instead of 9hr 

CNN_accum_24, inds_start, inds_end = utils.accum_slide_window(CNN_output, accum_window, output_freq, skip_start)
CNN_accum_24[..., land_mask_bc] = np.nan

LEADs_24H_hrs = LEADs_3H_hrs[inds_end]

GEFS_accum_24, inds_start, inds_end = utils.accum_slide_window(GEFS_raw, accum_window, output_freq, skip_start)
GEFS_accum_24[..., land_mask_bc] = np.nan

DATA['CNN_24_P10'] = np.quantile(CNN_accum_24, 0.1, axis=0)
DATA['CNN_24_P50'] = np.quantile(CNN_accum_24, 0.5, axis=0)
DATA['CNN_24_P90'] = np.quantile(CNN_accum_24, 0.9, axis=0)

DATA['GEFS_24_P10'] = np.quantile(GEFS_accum_24, 0.1, axis=0)
DATA['GEFS_24_P50'] = np.quantile(GEFS_accum_24, 0.5, axis=0)
DATA['GEFS_24_P90'] = np.quantile(GEFS_accum_24, 0.9, axis=0)


In [16]:
accum_window = 24 # 24x3h = 3 day
output_freq = 2 # 2x3h = 6h per output
skip_start = 1 # skip the first 3hr, start from 12hr instead of 9hr 

CNN_accum_72, inds_start, inds_end = utils.accum_slide_window(CNN_output, accum_window, output_freq, skip_start)
CNN_accum_72[..., land_mask_bc] = np.nan

LEADs_72H_hrs = LEADs_3H_hrs[inds_end]

GEFS_accum_72, inds_start, inds_end = utils.accum_slide_window(GEFS_raw, accum_window, output_freq, skip_start)
GEFS_accum_72[..., land_mask_bc] = np.nan

DATA['CNN_72_P10'] = np.quantile(CNN_accum_72, 0.1, axis=0)
DATA['CNN_72_P50'] = np.quantile(CNN_accum_72, 0.5, axis=0)
DATA['CNN_72_P90'] = np.quantile(CNN_accum_72, 0.9, axis=0)

DATA['GEFS_72_P10'] = np.quantile(GEFS_accum_72, 0.1, axis=0)
DATA['GEFS_72_P50'] = np.quantile(GEFS_accum_72, 0.5, axis=0)
DATA['GEFS_72_P90'] = np.quantile(GEFS_accum_72, 0.9, axis=0)

In [17]:
len(LEADs_72H_hrs)

15

In [18]:

             
# def precip_map(data_pair, lon, lat, lead_hrs, accum_hrs, dt_utc_now, 
#                cmap_precip, label_precip, linewidth_map, Px_str, accum_str,
#                edge, center_lon, shape_watershed_dir, PROVINCE, geom_US, fig_keys, png_bch_name, scale_param='50m', font_text=14, ):
#     '''
#     xxx
#     '''
#     vmin = label_precip[0]
#     vmax = label_precip[-1]
#     N_colors = len(label_precip)-1 # color = label-1

#     dt_ini_str = datetime.strftime(dt_utc_now, '%Y-%h-%d %HZ')
    
#     TAGs = ['raw', 'post-processed']
    

#     accum_ = png_bch_name['accum_'].format(str(accum_hrs))
#     dt_fmt = datetime.strftime(dt_utc_now, png_bch_name['dt_fmt_'])
    
#     if center_lon == -125:
#         region = 'swbc'
#     else:
#         region = 'bc'
    
#     TITLE_base = png_bch_name['base_'].format(accum_, region, dt_fmt)+png_bch_name['tail']
    
#     for lead, fcst_h in enumerate(lead_hrs):
        
#         TITLE_text = TITLE_base.format(int(fcst_h))
#         handle_title = []
#         # --------------------------------------------- #
#         # Calculate & format datetime
#         dt_valid = dt_utc_now+timedelta(hours=fcst_h)
#         dt_accum_start = dt_valid-timedelta(hours=accum_hrs)

#         dt_valid_str = datetime.strftime(dt_valid, '%Y-%h-%d %HZ')
#         dt_accum0_str = datetime.strftime(dt_accum_start, '%Y-%h-%d %HZ')
#         dt_accum1_str = datetime.strftime(dt_valid, '%h-%d %HZ')

#         h_str = ' ({} hrs)'.format(int(fcst_h))

#         fig = plt.figure(figsize=(1.25*15, 1.25*8), dpi=fig_keys['dpi'])
#         gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])
        
#         proj_ = ccrs.NorthPolarStereo(central_longitude=center_lon)
        
#         ax1 = plt.subplot(gs[0, 0], projection=proj_)
#         ax2 = plt.subplot(gs[0, 1], projection=proj_)
#         plt.subplots_adjust(0, 0, 1, 1, hspace=0, wspace=0)
        
#         AX = [ax1, ax2]
        
#         TEXT_poos1 = [[0.010, 0.248, 0.20, 0.0450], [0.510, 0.248, 0.20, 0.0450]]
#         TEXT_poos2 = [[0.020, 0.198, 0.20, 0.0450], [0.520, 0.198, 0.20, 0.0450]]
#         TEXT_poos3 = [[0.020, 0.178, 0.26, 0.0225], [0.520, 0.178, 0.26, 0.0225]]
        
#         for i, ax in enumerate(AX):
#             if i == 0:
#                 ax.set_title(TITLE_text, x=0.7, ha='left', fontsize=font_text+4)
#             # --------------------------------------------- #
#             # Map configuration
#             ax.set_extent(edge, ccrs.PlateCarree())

#             ax.add_feature(cfeature.LAND.with_scale(scale_param), facecolor='none', zorder=1)
#             ax.add_feature(cfeature.COASTLINE.with_scale(scale_param), edgecolor='k', linewidth=linewidth_map, zorder=6)
#             ax.add_feature(cfeature.BORDERS.with_scale(scale_param), linestyle='--', linewidth=linewidth_map, zorder=6)
#             ax.add_feature(PROVINCE, edgecolor='k', linestyle=':', linewidth=linewidth_map, zorder=5)

#             ax.add_geometries(Reader(shape_watershed_dir).geometries(), ccrs.PlateCarree(),
#                               facecolor='none', edgecolor='0.25', linewidth=linewidth_map, hatch='//', zorder=3) #3
#             ax.add_geometries(geom_US, ccrs.PlateCarree(),
#                               facecolor='w', edgecolor='none', linewidth=0, zorder=4)
#             ax.spines['geo'].set_linewidth(2.5)
#             ax.spines['geo'].set_zorder(9)
            
#             # --------------------------------------------- #
            
#             data = data_pair[i]
#             tag = TAGs[i]
            
#             CS_ = ax.contourf(lon, lat, data[lead, ...], cmap=cmap_precip, extend='max', transform=ccrs.PlateCarree(),  
#                               levels=label_precip, norm=mcolors.BoundaryNorm(label_precip, ncolors=N_colors), zorder=2)

#             CS_ = ax.contourf(lon[24:, :46], lat[24:, :46], data[lead, 24:, :46], cmap=cmap_precip, extend='max', 
#                               transform=ccrs.PlateCarree(), levels=label_precip, 
#                               norm=mcolors.BoundaryNorm(label_precip, ncolors=N_colors), zorder=5)

#             ax.contour(lon, lat, data[lead, ...], levels=label_precip, colors=('0.5',), linewidths=(linewidth_map,), extend='max', 
#                        transform=ccrs.PlateCarree(), norm=mcolors.BoundaryNorm(label_precip, ncolors=N_colors), zorder=2)
            
    
#             # --------------------------------------------- #
#             # Title and text boxes

#             ax_t1 = fig.add_axes(TEXT_poos1[i], facecolor='w')
#             [j.set_linewidth(0.0) for j in ax_t1.spines.values()]
#             ax_t1.tick_params(axis='both', left=False, top=False, right=False, bottom=False, 
#                               labelleft=False, labeltop=False, labelright=False, labelbottom=False)

#             handle_title += plib.string_partial_format(fig, ax_t1, 0, 1, 'left', 'top', 
#                                                   ['The ', Px_str, ' percentile ', tag], 
#                                                   ['k',]*4, [font_text,]*4, ['normal', 'bold', 'normal', 'bold'])
#             handle_title += plib.string_partial_format(fig, ax_t1, 0, 0.5, 'left', 'top', 
#                                                   ['GEFS ', accum_str, ' total precip ensemble'], 
#                                                   ['k',]*3, [font_text,]*3, ['normal', 'bold', 'normal'])

#             ax_t2 = fig.add_axes(TEXT_poos2[i], facecolor='w')
#             [j.set_linewidth(0.0) for j in ax_t2.spines.values()]
#             ax_t2.tick_params(axis='both', left=False, top=False, right=False, bottom=False, 
#                               labelleft=False, labeltop=False, labelright=False, labelbottom=False)

#             handle_title += plib.string_partial_format(fig, ax_t2, 0, 1, 'left', 'top', 
#                                                   ['Initi time: ', dt_ini_str], 
#                                                   ['k',]*2, [font_text,]*2, ['normal', 'normal'])
#             handle_title += plib.string_partial_format(fig, ax_t2, 0, 0.525, 'left', 'top', 
#                                                   ['Valid time: ', dt_valid_str, h_str], 
#                                                   ['k',]*3, [font_text,]*3, ['normal', 'normal', 'normal'])
        
        
#             ax_t3 = fig.add_axes(TEXT_poos3[i], facecolor='w')
#             [j.set_linewidth(0.0) for j in ax_t3.spines.values()]
#             ax_t3.tick_params(axis='both', left=False, top=False, right=False, bottom=False, 
#                               labelleft=False, labeltop=False, labelright=False, labelbottom=False)
        
#             handle_title += plib.string_partial_format(fig, ax_t3, 0, 1.0, 'left', 'top', 
#                                                   ['Accum time: from ', dt_accum0_str, ' to ', dt_accum1_str], 
#                                                   ['k',]*4, [font_text,]*4, ['normal', 'normal', 'normal', 'normal'])
        
#         for handle in handle_title:
#             handle.set_bbox(dict(facecolor='w', edgecolor='none', pad=0.0, zorder=6))
        
#         ax_base = fig.add_axes([1.01, 0.2, 0.065, 0.6], facecolor='none')
#         [j.set_linewidth(0) for j in ax_base.spines.values()]
#         ax_base.tick_params(axis='both', left=False, top=False, right=False, bottom=False, 
#                             labelleft=False, labeltop=False, labelright=False, labelbottom=False)
#         cax = inset_axes(ax_base, height='100%', width='25%', borderpad=0, loc=2)
#         CBar = plt.colorbar(CS_, orientation='vertical', extend='max', ticks=label_precip, cax=cax)
#         CBar.ax.tick_params(axis='y', labelsize=font_text, direction='in', length=21)
#         CBar.set_label('[mm]', fontsize=font_text)
#         CBar.outline.set_linewidth(2.5)

In [19]:
camp_precip, label_precip = plib.precip_cmap(return_label=True, accum_map=False)
lon = lon_bc
lat = lat_bc

In [35]:
#MODELs = ['CNN', 'GEFS']
#TAGs = ['post-processed', 'raw']
# model = 'CNN'
# tag = 'post-processed'

ACCUMs = ['72',] #'3', '24', 
ACCUM_strs = ['3 days',] #'3 hourly', '1 day', 
ACCUM_flags = [True,] #False, True, 
LEADs = (LEADs_72H_hrs,) #LEADs_3H_hrs, LEADs_24H_hrs, 

Ps = ['P90',] #'P10', 'P50', 
P_strs = ['90-th',] #'10-th', '50-th', 

EDGEs = [edge_bc, edge_sw]
CENTERs = [center_lon_bc, center_lon_sw]

    
for j, accum in enumerate(ACCUMs[:1]):
    accum_hrs = int(accum)
    accum_str = ACCUM_strs[j]
    lead_hrs = LEADs[j]
    cmap_precip, label_precip = plib.precip_cmap(return_label=True, accum_map=ACCUM_flags[j])

    for k, P in enumerate(Ps[:1]):
        P_str = P_strs[k]

        data_pair = (DATA['GEFS_{}_{}'.format(accum, P)], DATA['CNN_{}_{}'.format(accum, P)])
        
        for l, edge in enumerate(EDGEs):
            center_lon = CENTERs[l]
            
            plib.precip_map(data_pair, lon, lat, lead_hrs, accum_hrs, dt_utc_now, 
                            camp_precip, label_precip, linewidth_map, P_str, accum_str, 
                            edge, center_lon, shape_watershed_dir, PROVINCE, geom_US, fig_keys, png_bch_name, scale_param='50m', font_text=14)

            